# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [67]:
import numpy as np

def get_threshold(feature, y, train_mode='default'):
    best_thr = None
    best_q = float('inf')
    arr = np.argsort(feature)
    left_el = 0
    right_el = y.shape[0]
    left_sum = 0
    right_sum = 0
    left_sum_sq = 0
    right_sum_sq = 0
    for index in arr:
        right_sum += y[index]
        right_sum_sq += y[index] ** 2
    
    for i in range(len(arr) - 1):
        index = arr[i]
        left_sum += y[index]
        right_sum -= y[index]
        left_el += 1
        right_el -= 1
        left_sum_sq += y[index] ** 2
        right_sum_sq -= y[index] ** 2
        left_dis = left_sum_sq - left_sum ** 2 / left_el
        right_dis = right_sum_sq - right_sum ** 2 / right_el
        if (left_dis + right_dis) < best_q:
            best_q = left_dis + right_dis
            best_thr = feature[index]
            left_mean = left_sum / left_el
            right_mean = right_sum / right_el
        
    return best_q, best_thr, left_mean, right_mean


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr, left, right = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
            left_mean = left
            right_mean = right
    return best_f, best_thr, left_mean, right_mean

def predict(X, best_f, best_thr, left_mean, right_mean):
    pred = []
    for elem in X:
        if elem[best_f] <= best_thr:
            pred.append(left_mean)
        else:
            pred.append(right_mean)
    return pred

In [2]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [66]:
%%time

best_f, best_thr, left_mean, right_mean = train_stump(X, y)
pred = predict(X, best_f, best_thr, left_mean, right_mean)
print(mean_squared_error(y, pred))

!!! 34450.122685406684
!!! 31633.069947244992
!!! 23376.740388616818
46.19909167710848
Wall time: 44.9 ms
